# SQL Extra Topics - SQLAlchemy

The SQLAlchemy SQL Toolkit and Object Relational Mapper is a comprehensive set of tools for working with databases and Python

We'll need:

- [Azure SQL Database](https://portal.azure.com/) --> tenemos que configurar la base de datos

- [ODBC Driver for SQL Server](https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15) --> Tenemos que instalar el driver

- [pyodbc](https://pypi.org/project/pyodbc/) --> moulo necesario para instalar en la terminal para poder usar sqlalchemy y tirar queries a un database (HECHO)

- [SQLAlchemy](https://docs.sqlalchemy.org/en/13/intro.html)

In [1]:
# imports

import os
from dotenv import load_dotenv
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import pandas as pd

sqlalchemy.__version__

'1.4.29'

---

### dotenv

https://pypi.org/project/python-dotenv/

In [2]:
load_dotenv('.env') #para poder leer contraseñas que tenemos en un archivo dot.env en local

DATABASE_PASSWORD = os.environ.get("DATABASE_PASSWORD")
DATABASE_PASSWORD

---

In [20]:
# connection string params => https://docs.sqlalchemy.org/en/14/dialects/mssql.html#pass-through-exact-pyodbc-string
#está en el menu "cadena de conexion"

driver = 'Driver={ODBC Driver 17 for SQL Server};'
server = 'Server=tcp:sqlass123.database.windows.net,1433;' #en vez de sqlironhack es el de nustras iniciales "sqlass123"
database = 'Database=mySampleDatabase;'
uid = 'Uid=azureuser;'
#pwd = f'Pwd={DATABASE_PASSWORD};'
pwd = 'Pwd=P@~~w0rd123;'
config = 'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

In [21]:
#EN ESTA CUENTA SOLO ME PUEDO CONECTAR DESDE LA ESCUELA DE IRONHACK POR LA IP

#driver = 'Driver={ODBC Driver 17 for SQL Server};'
#server = 'Server=tcp:sqlironhack.database.windows.net,1433;'
#database = 'Database=AdventureWorksLT;'
#uid = 'Uid=azureuser;'
#pwd = f'Pwd={DATABASE_PASSWORD};'
#pwd = 'Pwd=P@$$w0rd123;'
#config = 'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

In [22]:
# connection string assembly

connection_string = driver+server+database+uid+pwd+config 
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string}) #"mssql+pyodbc" = motor de busqueda de la bbdd + driver para importar
connection_url

mssql+pyodbc://?odbc_connect=Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BServer%3Dtcp%3Asqlass123.database.windows.net%2C1433%3BDatabase%3DmySampleDatabase%3BUid%3Dazureuser%3BPwd%3DP%40~~w0rd123%3BEncrypt%3Dyes%3BTrustServerCertificate%3Dno%3BConnection+Timeout%3D30%3B

In [23]:
# SQL Engine => dialect+driver (https://docs.sqlalchemy.org/en/14/core/engines.html)

engine = create_engine(connection_url)
type(engine)

sqlalchemy.engine.base.Engine

---

In [24]:
# Pandas!!!

df = pd.read_sql_query("SELECT * FROM SalesLT.ProductModel", engine)
df

,ProductModelID,Name,CatalogDescription,rowguid,ModifiedDate
0,1,Classic Vest,None,29321D47-1E4C-4AAC-887C-19634328C25E,2007-06-01
1,2,Cycling Cap,None,474FB654-3C96-4CB9-82DF-2152EEFFBDB0,2005-06-01
2,3,Full-Finger Gloves,None,A75483FE-3C47-4AA4-93CF-664B51192987,2006-06-01
3,4,Half-Finger Gloves,None,14B56F2A-D4AA-40A4-B9A2-984F165ED702,2006-06-01
4,5,HL Mountain Frame,None,FDD5407B-C2DB-49D1-A86B-C13A2E3582A2,2005-06-01
...,...,...,...,...,...
123,124,ML Mountain Rear Wheel,None,D968D774-778E-4399-A3C5-375176418229,2006-06-01
124,125,HL Mountain Rear Wheel,None,95450545-ADF7-48F3-899E-964DE8920DC6,2006-06-01
125,126,LL Road Rear Wheel,None,95946BD4-C6D9-4344-8066-317D8957EA21,2006-06-01
126,127,Rear Derailleur,None,F9327E5D-F8B6-40C5-BFA9-63F886BDFC24,2007-06-01


---

### Let's see some examples...

In [25]:
query_1 = '''
SELECT TOP 20 pc.Name as CategoryName, p.name as ProductName 
FROM SalesLT.ProductCategory pc JOIN SalesLT.Product p ON pc.productcategoryid = p.productcategoryid
ORDER BY CategoryName
'''

In [26]:
df_q1 = pd.read_sql_query(query_1, engine)
df_q1

,CategoryName,ProductName
0,Bib-Shorts,"Men's Bib-Shorts, S"
1,Bib-Shorts,"Men's Bib-Shorts, M"
2,Bib-Shorts,"Men's Bib-Shorts, L"
3,Bike Racks,Hitch Rack - 4-Bike
4,Bike Stands,All-Purpose Bike Stand
5,Bottles and Cages,Water Bottle - 30 oz.
6,Bottles and Cages,Mountain Bottle Cage
7,Bottles and Cages,Road Bottle Cage
8,Bottom Brackets,LL Bottom Bracket
9,Bottom Brackets,ML Bottom Bracket


In [27]:
query_2 = '''
SELECT c.FirstName + ' ' + c.LastName AS [Customer Fullname], p.Name AS [Product Name]
FROM SalesLT.Customer AS c
    INNER JOIN SalesLT.SalesOrderHeader AS soh ON c.CustomerID = soh.CustomerID
    INNER JOIN SalesLT.SalesOrderDetail AS shd ON soh.SalesOrderID = shd.SalesOrderID
    INNER JOIN SalesLT.Product AS p ON shd.ProductID = p.ProductID
ORDER BY [Customer Fullname], [Product Name]
'''

In [28]:
df_q2 = pd.read_sql_query(query_2, engine)
df_q2

,Customer Fullname,Product Name
0,Andrea Thomsen,Rear Brakes
1,Anthony Chor,"HL Touring Frame - Blue, 50"
2,Anthony Chor,"HL Touring Frame - Blue, 54"
3,Anthony Chor,"HL Touring Frame - Blue, 60"
4,Anthony Chor,"HL Touring Frame - Yellow, 60"
...,...,...
537,Walter Mays,"Racing Socks, M"
538,Walter Mays,"Road-350-W Yellow, 40"
539,Walter Mays,"Road-750 Black, 48"
540,Walter Mays,"Road-750 Black, 52"


In [29]:
query_3 = '''
SELECT   p.name, COUNT(*) AS 'Total Orders'
FROM SalesLT.Product AS p
    INNER JOIN SalesLT.SalesOrderDetail AS sod ON p.ProductID = sod.ProductID
GROUP BY  p.Name
ORDER BY 'Total Orders' DESC
'''

In [30]:
df_q3 = pd.read_sql_query(query_3, engine)
df_q3

,name,Total Orders
0,"Classic Vest, S",10
1,"Long-Sleeve Logo Jersey, L",10
2,AWC Logo Cap,9
3,"Short-Sleeve Classic Jersey, XL",9
4,"Short-Sleeve Classic Jersey, L",8
...,...,...
137,"Road-750 Black, 44",1
138,"Road-350-W Yellow, 44",1
139,Rear Brakes,1
140,"Touring-3000 Blue, 62",1


---